<a href="https://colab.research.google.com/github/ColonelBBQ/PCC_2022/blob/main/B2C_Conversion_Emulators_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
if __name__ == "__main__":
    import itertools
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import csv
    from tqdm.notebook import tqdm, trange
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    import matplotlib.pyplot as plt 
    import seaborn as sns
    from scipy.stats import pearsonr
    from sklearn import linear_model
    from scipy import stats
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    import seaborn as sns
    from dateutil.relativedelta import relativedelta
    from datetime import date
    from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
    from functools import reduce
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_classif, mutual_info_classif, chi2
    pd.set_option('display.max_colwidth', None)
    from sklearn.model_selection import train_test_split
    from sklearn.utils import resample
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from scipy.special import expit
    from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import confusion_matrix

    import pandas as pd
    from pandas.io import gbq
    import datetime, calendar
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 100)
    def style_negative(v, props=''):
        return props if v < 0 else None

    
    sql = """
    SELECT contract_reference, business_context_business_provider_code,business_unit_code, 
    partner_creditbureau_crif_last_call_perform_score,contact_public_current_address_province_code,
    contact_private_declared_birth_date,situation_declared_number_of_paid_months_per_year, 
    contact_public_declared_nationality_code, contact_public_declared_address_town_label, 
    contact_public_declared_province_code, situation_declared_profession_code ,
    situation_declared_main_net_monthly_income, situation_verified_main_net_monthly_income, 
    product_loan_initial_demand_maturity, product_loan_initial_demand_requested_amount, 
    situation_declared_professional_situation_code, situation_declared_profession_activity_area_code, 
    product_loan_initial_demand_project_type_code, process_decisions_timeline_application_date, 
    process_decisions_timeline_pre_decision_status, process_decisions_timeline_decision_status,
    partner_creditbureau_experian_last_call_delphi_g3_index1,partner_creditbureau_experian_last_call_delphi_g3_index3,
    partner_creditbureau_experian_last_call_cdi,situation_declared_profession_start_date,
    product_loan_selected_application_fee_rate,product_loan_selected_lending_rate, 
    product_loan_selected_application_fee_rate, 
    product_loan_selected_nominal_annual_rate, product_loan_selected_annual_percentage_rate,process_decisions_credit_scoring_declared_score_value 
    FROM `yuc-data.private_datalake_ww.all_flat` 
    where business_unit_code = 'IT' and 
        process_decisions_timeline_application_date >= '2022-01-01' and 
        business_context_business_provider_code in (
        "FACILE",
        "FACILE-PROMO",
        "FACILE-PROMO-2",
        "PRESTITIONLINE",
        "PRESTITIONLINE-PROMO",
        "PRESTITIONLINE-PROMO-2")

    """
    df = gbq.read_gbq(sql, project_id = "yuc-data")
    
    sql = """
    SELECT 
    B2C.contract_reference, 
    B2C.verified_situation.ongoing_credit_monthly_amount as credit,
    B2C.verified_situation.mortgage_amount as mortgage, 
    B2C.verified_situation.rent_amount as rent
    FROM `yuc-pr-dataanalyst`.`intermediate`.`transfo_b2c_classic` as B2C 
    WHERE B2C.decision_timeline.decision_status = 'granted' AND 
          B2C.business_context.business_unit_code = 'IT'
    """
    df_creditOngoing = gbq.read_gbq(sql, project_id = "yuc-data")
    df_creditOngoing = df_creditOngoing.fillna(0)
    print('reading query . . .')
    df = pd.merge(df, df_creditOngoing, on='contract_reference', how='left')
    df['application_date'] = pd.to_datetime(df['process_decisions_timeline_application_date']).dt.normalize()
    df['application_date'] = df.application_date.dt.tz_convert(None).dt.round('D')
    df['application_date_yymm']=df['application_date'].dt.strftime('%Y-%m')
    df['contact_private_declared_birth_date'] = pd.to_datetime(df['contact_private_declared_birth_date']).dt.normalize()
    df['situation_declared_profession_start_date'] = pd.to_datetime(df['situation_declared_profession_start_date']).dt.normalize()
 

    import time
    import datetime
    df['application_date_in_quarter'] = df['application_date'].dt.to_period('Q')


    Corrected_Income = []
    for x,y in zip(df.situation_declared_main_net_monthly_income,df.situation_declared_number_of_paid_months_per_year):
        try:
            Corrected_Income.append(float(y[1:])*x/12)
        except:
            Corrected_Income.append(x)
    df['Corrected_Income'] = np.array(Corrected_Income)

    #-------------------------------------------------------------------------------
    Crif_score_GRP = []
    for x in df['partner_creditbureau_crif_last_call_perform_score']:
            if x in ['A','B','C','D']:
                Crif_score_GRP.append('KO')
            elif x in ['E','F','G']:
                Crif_score_GRP.append('Risk')
            elif x in ['H','I','L']:
                Crif_score_GRP.append('Medium')
            elif x in ['M','N',"O","P"]:
                Crif_score_GRP.append('Safe')                       
            else:
                Crif_score_GRP.append('Missing')
    df['Crif_score_GRP'] = Crif_score_GRP
    print(df['Crif_score_GRP'].value_counts())

    #-------------------------------------------------------------------------------
    Experian_score_GRP = []
    for x,cdi in zip(df['partner_creditbureau_experian_last_call_delphi_g3_index1'],df['partner_creditbureau_experian_last_call_cdi']):
        try:
            if cdi == 25:
                Experian_score_GRP.append('No hit')
            elif x in [3, 4, 5, 6, 7, 8, 9]:
                Experian_score_GRP.append('Risk')
            elif x in [10, 11, 12, 13, 14]:
                Experian_score_GRP.append('Medium')
            elif x in [15,16, 17, 18, 19, 20]:
                Experian_score_GRP.append('Safe')
            else: 
                Experian_score_GRP.append('KO_Before')
        except:
            Experian_score_GRP.append('Missing')
    df['Experian_score_GRP'] = Experian_score_GRP
    print(df['Experian_score_GRP'].value_counts())

    Income_GRP = []
    for x in df['Corrected_Income']:
            if 800 <= x < 1000:
                Income_GRP.append('[800-1000[')
            elif 1000<= x < 1400:
                Income_GRP.append('[1000-1400[')
            elif 1400<= x < 1800:
                Income_GRP.append('[1400-1800[')
            elif x >= 1800:
                Income_GRP.append('>1800')
            else:
                Income_GRP.append('Missing')
    df['Income_GRP'] = Income_GRP
    print(df['Income_GRP'].value_counts())
    print()
        


#     Cashflow = []
#     for credit,mortgage,rent,rata,income in zip(df.credit,df.mortgage,df.rent,df.product_loan_initial_demand_requested_amount,df.Corrected_Income):
#         tot = income-(credit+mortgage+rent+rata)
#         Cashflow.append(tot)
#     df['Cashflow'] = np.array(Cashflow)

#     Cashflow_GRP = []
#     for x in df.Cashflow:
#             if  500<= x < 525:
#                 Cashflow_GRP.append('[500-525[')
#             elif 525<= x < 600:
#                 Cashflow_GRP.append('[525-600[')
#             elif x >= 600:
#                 Cashflow_GRP.append('>=600')
#             else:
#                 Cashflow_GRP.append('Missing')        
#     df['Cashflow_GRP'] = np.array(Cashflow_GRP)  

#     Debtrate = []
#     for credit,mortgage,rata,income in zip(df.credit,df.mortgage,df.product_loan_initial_demand_requested_amount,df.Corrected_Income):
#         tot = (credit+mortgage+rata)/income
#         Debtrate.append(tot)
#     df['Debtrate'] = np.array(Debtrate)

#     Debtrate_GRP = []
#     for x in df.Debtrate:
#             if  x < 0.2:
#                 Debtrate_GRP.append('[0-20%[')
#             elif 0.2<= x < 0.4:
#                 Debtrate_GRP.append('[20%-40%[')
#             elif 0.4<= x < 0.6:
#                 Debtrate_GRP.append('[40%-60%[')            
#             elif 0.6<= x < 0.8:
#                 Debtrate_GRP.append('[60%-80%[')            
#             elif x >= 0.8:
#                 Debtrate_GRP.append('>=80%')
#             else:
#                 Debtrate_GRP.append('Missing')        
#     df['Debtrate_GRP'] = np.array(Debtrate_GRP)    


        
    list_ = []
    for x in df.contact_public_current_address_province_code:
        if x in ["CH","AQ","PE","TE"]:
            list_.append('Abruzzo')
        elif x in ["MT","PZ",]:
            list_.append("Basilicata")
        elif x in ["CZ","CS","KR","RC","VV"]:
            list_.append("Calabria")
        elif x in ["AV","BN","CE","NA","SA"]:
            list_.append("Campania")
        elif x in ["BO","FE","FC","MO","PR","PC","RA","RE","RN"]:
            list_.append("Emilia-Romagna")
        elif x in ["GO","PN","TS","UD"]:
            list_.append("Friuli-Venezia Giulia")
        elif x in ["FR","LT","RI","RM","VT"]:
            list_.append("Lazio")
        elif x in ["GE","IM","SP","SV"]:
            list_.append("Liguria")
        elif x in ["BG","BS","CO","CR","LC","LO","MN","MI","MB","PV","SO","VA"]:
            list_.append("Lombardia")
        elif x in ["AN","AP","FM","MC","PU"]:
            list_.append("Marche")
        elif x in ["CB","IS"]:
            list_.append("Molise")
        elif x in ["AL","AT","BI","CN","NO","TO","VB","VC"]:
            list_.append("Piemonte")
        elif x in ["BA","BT","BR","FG","LE","TA"]:
            list_.append("Puglia")
        elif x in ["CA","CI","NU","OT","OR","VS","SS","OG"]:
            list_.append("Sardegna")
        elif x in ["AG","CL","CT","EN","ME","PA","RG","SR","TP"]:
            list_.append("Sicilia")
        elif x in ["AR","FI","GR","LI","LU","MS","PI","PT","PO","SI"]:
            list_.append("Toscana")
        elif x in ["BZ","TN"]:
            list_.append("Trentino-Alto Adige")
        elif x in ["PG","TR"]:
            list_.append("Umbria")
        elif x in ["AO"]:
            list_.append("Valle d'Aosta")        
        elif x in ["BL","PD","RO","TV","VE","VR","VI"]:
            list_.append("Veneto")
        else:
            list_.append('Missing')
    df['Region'] = np.array(list_)        

    list_ = []
    for x in df.Region:
        if x in ["Marche","Toscana","Emilia-Romagna","Abruzzo","Lazio","Umbria"]:
            list_.append("CE")
        elif x in ["Sicilia","Sardegna"]:
            list_.append("IS")  
        elif x in ["Veneto","Trentino-Alto Adige","Friuli-Venezia Giulia"]:
            list_.append("NE") 
        elif x in ["Piemonte","Valle d'Aosta","Lombardia","Liguria"]:
            list_.append("NO") 
        elif x in ["Campania","Puglia","Molise","Calabria","Basilicata"]:
            list_.append("SU")
        else:
            list_.append('Missing')
    df['Rip_geo'] = np.array(list_)
    df['Rip_geo'].value_counts()        

    list_ = []
    for x in df.product_loan_initial_demand_maturity:
        try:
            if x < 36:
                list_.append('<36')
            elif x < 60:
                list_.append('[36-48]')
            elif x <= 72:
                list_.append('[60-72]')
            elif x <= 84:
                list_.append('[72-84]')
            else:
                list_.append('Missing')
        except:
            list_.append('Missing')
    df['Maturity_GRP'] = list_
    df['Maturity_GRP'].value_counts()

    Borrowed_Amount_GRP = []
    for x in df['product_loan_initial_demand_requested_amount']:
            if 1000 <= x < 5000:
                Borrowed_Amount_GRP.append('[1000-5000[')
            elif 5000 <= x < 10000:
                Borrowed_Amount_GRP.append('[5000-10000[')
            elif 10000 <= x < 15000:
                Borrowed_Amount_GRP.append('[10000-15000[')
            elif 15000 <= x < 20000:
                Borrowed_Amount_GRP.append('[15000-20000[')
            elif 20000 <= x < 30000:
                Borrowed_Amount_GRP.append('[20000-30000[')      
            elif x >= 30000:
                Borrowed_Amount_GRP.append('[30000-50000]')
            else:
                Borrowed_Amount_GRP.append('Missing')
    df['Borrowed_Amount_GRP'] = Borrowed_Amount_GRP
    print(df['Borrowed_Amount_GRP'].value_counts())
    print()

        


    #------- GRP for Age -------
    now = date.today()
    age = []
    for age_,appl in zip(df['contact_private_declared_birth_date'],df['application_date']):
        try:
            rdelta = relativedelta(appl, age_)
            diff_year = rdelta.years
            age.append(diff_year)
        except:
            age.append(np.nan)
    df['AGE'] = age


    Age_GRP = []
    for x in df['AGE']:
        try:
            if 0<=x<31:
                Age_GRP.append('[21-31[')
            elif 31<=x<51:
                Age_GRP.append('[31-51[')
            elif x>=51:
                Age_GRP.relaappend('>=51')
            else:
                Age_GRP.append('Missing')
        except:
            Age_GRP.append('Missing')
    df['Age_GRP'] = Age_GRP
    print(df['Age_GRP'].value_counts())


    Cutoff = []
    for x in df['process_decisions_credit_scoring_declared_score_value']:
        try:
            if 0<=x<9518:
                Cutoff.append('A7')
            elif 9518<=x<9650:
                Cutoff.append('A6')
            elif 9650<=x<9700:
                Cutoff.append('A5')
            elif 9700<=x<9795:
                Cutoff.append('A4')
            elif 9795<=x<9830:
                Cutoff.append('A3')
            elif 9830<=x<9910:
                Cutoff.append('A2')
            elif 9910<=x<=10000:
                Cutoff.append('A1')
            else:
                Cutoff.append('Missing')
        except:
            Cutoff.append('Missing')
    df['Cutoff'] = np.array(Cutoff)
    df['Cutoff'].value_counts()

        
    Profession_mapping = []
    for x,y in zip(df.situation_declared_profession_code,df.situation_declared_professional_situation_code):
        if x in ['ARCHITECT','BUSINESSCONSULTANT','DOCTOR','ENGINEER','LAWYER','NOTARY']:
            Profession_mapping.append('LIBERALPROFESSION')
        else:
            Profession_mapping.append(y)
    df['Profession_mapping'] = Profession_mapping
    print(df['Profession_mapping'].value_counts())


    #------- GRP for Seniority -------
    Seniority = []
    for jobs,appl in zip(df['situation_declared_profession_start_date'],df['application_date']):
        try:
            rdelta = relativedelta(appl,jobs)
            diff_year = rdelta.years
            Seniority.append(diff_year)
        except:
            Seniority.append(np.nan)
    df['Seniority'] = Seniority
    df.Seniority.value_counts()

    # Questo mapping vale solo per PERMANENT, per gli independent il filtro minimo è 3 anni
    Seniority_GRP = []
    for x,profession in zip(df['Seniority'],df['Profession_mapping']):
        if profession == 'INDEPENDENT': 
            try:
                if 0<=x<5:
                    Seniority_GRP.append('[3-5[ I')
                elif 5<=x<7:
                    Seniority_GRP.append('[5-7[ I')
                elif 7<=x<10:
                    Seniority_GRP.append('[7-10[ I')
                elif x>=10:
                    Seniority_GRP.append('>=10 I')                
                else:
                    Seniority_GRP.append('Missing I')
            except:
                Seniority_GRP.append("Missing I")
        elif profession == 'PERMANENT':
            try:
                if 0<=x<2:
                    Seniority_GRP.append('[0-2[ P')
                elif 2<=x<4:
                    Seniority_GRP.append('[2-4[ P')
                elif 4<=x<8:
                    Seniority_GRP.append('[4-8[ P')
                elif x>=8:
                    Seniority_GRP.append('>=8 P')                
                else:
                    Seniority_GRP.append('Missing P')
            except:
                Seniority_GRP.append('Missing P')
        elif profession != 'PERMANENT' or profession != 'INDEPENDENT'  :
            try:
                if 0<=x<2:
                    Seniority_GRP.append('[0-2[ others')
                elif 2<=x<4:
                    Seniority_GRP.append('[2-4[ others')
                elif 4<=x<8:
                    Seniority_GRP.append('[4-8[ others')
                elif x>=8:
                    Seniority_GRP.append('>=8 others')                
                else:
                    Seniority_GRP.append('Missing others')                    
            except:
                Seniority_GRP.append('Missing others')            
    df['Seniority_GRP'] = Seniority_GRP
    print(df['Seniority_GRP'].value_counts())
    
    list_ = []
    for delphi,cdi in zip(df.partner_creditbureau_experian_last_call_delphi_g3_index1,df.partner_creditbureau_experian_last_call_cdi):
        try:
            if delphi == 12 and cdi == 25:
                list_.append(3)
            elif delphi in [1,2]: 
                list_.append(1)
            elif delphi in [3,4]: 
                list_.append(2)            
            elif delphi in [5]: 
                list_.append(4)
            elif delphi in [6]: 
                list_.append(5)
            elif delphi in [7,8]: 
                list_.append(6)
            elif delphi in [9]: 
                list_.append(7)            
            elif delphi in [10]: 
                list_.append(8)            
            elif delphi in [11,12]: 
                list_.append(9)            
            elif delphi in [13,14,15,16,17,18,19,20]: 
                list_.append(10)    
            else: 
                list_.append('Missing')
        except: list_.append('Missing')
    df['quality_score'] = np.array(list_)
    list_ = []
    for x in df['process_decisions_timeline_decision_status']:
        if x == 'granted':
            list_.append(1)
        else:
            list_.append(0)

    df['conversion'] = list_
    print(df['conversion'].value_counts())
    print()

    
    Cutoff = []
    for x in df['process_decisions_credit_scoring_declared_score_value']:
        try:
            if 0<=x<9650:
                Cutoff.append('A7')
            elif 9650<=x<9690:
                Cutoff.append('A6')
            elif 9690<=x<9740:
                Cutoff.append('A5')
            elif 9740<=x<9829:
                Cutoff.append('A4')
            elif 9829<=x<9864:
                Cutoff.append('A3')
            elif 9864<=x<9954:
                Cutoff.append('A2')
            elif 9954<=x<=10000:
                Cutoff.append('A1')
            else:
                Cutoff.append('Missing')
        except:
            Cutoff.append('Missing')
    df['Cutoff_new'] = np.array(Cutoff)
    df['Cutoff_new'].value_counts()    
    
    
    print('\n completed')

reading query . . .
Medium     12770
Safe       12699
Missing     9531
Risk        7648
KO          2772
Name: Crif_score_GRP, dtype: int64
Medium       13764
Risk         11022
No hit        8390
Safe          7429
Missing       4693
KO_Before      122
Name: Experian_score_GRP, dtype: int64
>1800          17292
[1400-1800[    17251
[1000-1400[    10668
[800-1000[       198
Missing           11
Name: Income_GRP, dtype: int64

[5000-10000[     13738
[10000-15000[    12538
[1000-5000[       6314
[20000-30000[     5298
[15000-20000[     5176
>30000            2356
Name: Borrowed_Amount_GRP, dtype: int64

[31-51[    25857
Missing    12785
[21-31[     6778
Name: Age_GRP, dtype: int64
PERMANENT            36409
INDEPENDENT           4646
PENSIONER_RETIRED     3196
LIBERALPROFESSION     1169
Name: Profession_mapping, dtype: int64
>=8 P           14172
[0-2[ P          8328
[4-8[ P          6999
[2-4[ P          6910
>=8 others       2232
[3-5[ I          1997
>=10 I           1612
[4-8[ other

In [2]:


list_ = []
for cutoff,project,bpc in zip(df.Cutoff_new, df.product_loan_initial_demand_project_type_code,df.business_context_business_provider_code):
    if cutoff == 'A2' and project in ['LIQUIDITY','DEBTS'] and bpc == 'PRESTITIONLINE' :
        list_.append('A3')
    elif cutoff == 'A2' and project in ['LIQUIDITY','DEBTS'] and bpc == 'FACILE': 
        list_.append('A1')
    else:
        list_.append(cutoff)
df['Cutoff_new_v1'] = list_
df['Cutoff_new_v1'].value_counts()


A7         12719
Missing    12706
A4          4656
A1          4368
A2          4317
A3          3459
A5          1943
A6          1252
Name: Cutoff_new_v1, dtype: int64

In [13]:
list_ = []
for cutoff,project,bpc in zip(df.Cutoff_new, df.product_loan_initial_demand_project_type_code,df.business_context_business_provider_code):
    if cutoff == 'A1' and project in ['LIQUIDITY','DEBTS'] and bpc == 'PRESTITIONLINE' :
        list_.append(1)
    elif cutoff == 'A2' and project in ['LIQUIDITY','DEBTS'] and bpc == 'PRESTITIONLINE' :
        list_.append(1)
    elif cutoff == 'A3' and project in ['LIQUIDITY','DEBTS'] and bpc == 'PRESTITIONLINE' :
        list_.append(1)
    elif cutoff == 'A1' and bpc == 'FACILE' :
        list_.append(1)
    else:
        list_.append(0)

df['Macro_A'] = list_
df['Macro_A'].value_counts()

0    42023
1     3397
Name: Macro_A, dtype: int64

In [3]:
start_date='2022-11-01'
end_date = str(date.today())




In [16]:
df.head()

,contract_reference,business_context_business_provider_code,business_unit_code,partner_creditbureau_crif_last_call_perform_score,contact_public_current_address_province_code,contact_private_declared_birth_date,situation_declared_number_of_paid_months_per_year,contact_public_declared_nationality_code,contact_public_declared_address_town_label,contact_public_declared_province_code,situation_declared_profession_code,situation_declared_main_net_monthly_income,situation_verified_main_net_monthly_income,product_loan_initial_demand_maturity,product_loan_initial_demand_requested_amount,situation_declared_professional_situation_code,situation_declared_profession_activity_area_code,product_loan_initial_demand_project_type_code,process_decisions_timeline_application_date,process_decisions_timeline_pre_decision_status,process_decisions_timeline_decision_status,partner_creditbureau_experian_last_call_delphi_g3_index1,partner_creditbureau_experian_last_call_delphi_g3_index3,partner_creditbureau_experian_last_call_cdi,situation_declared_profession_start_date,product_loan_selected_application_fee_rate,product_loan_selected_lending_rate,product_loan_selected_application_fee_rate_1,product_loan_selected_nominal_annual_rate,product_loan_selected_annual_percentage_rate,process_decisions_credit_scoring_declared_score_value,credit,mortgage,rent,application_date,application_date_yymm,application_date_in_quarter,Corrected_Income,Crif_score_GRP,Experian_score_GRP,Income_GRP,Region,Rip_geo,Maturity_GRP,Borrowed_Amount_GRP,AGE,Age_GRP,Cutoff,Profession_mapping,Seniority,Seniority_GRP,quality_score,conversion,Cutoff_new,Macro_A
0,CIT20220719P98SMAS,PRESTITIONLINE,IT,M,PR,1946-11-02,13,ITALIAN,PARMA,PC,RETIREMENT,4000.0,4000.0,36,10000.0,PENSIONER_RETIRED,None,NEWCAR,2022-07-19 16:51:12+00:00,pre_approved,canceled,11,9,15,2003-04-01,0.0254,0.0406,0.0254,0.0399,0.0585,9920,NaN,NaN,NaN,2022-07-19,2022-07,2022Q3,4000.0,Safe,Medium,>1800,Emilia-Romagna,CE,[36-48],[10000-15000[,75,Missing,A1,PENSIONER_RETIRED,19,>=8 others,9,0,A2,0
1,CIT20220919AIIZIIA,FACILE,IT,I,TO,1998-09-24,13,ITALIAN,GRUGLIASCO,TO,WORKER,1080.0,1080.0,72,7000.0,INDEPENDENT,None,USEDCAR,2022-09-19 07:00:52+00:00,pre_approved,canceled,14,7,14,2019-07-01,0.0102,0.1534,0.0102,0.1436,0.1579,9553,NaN,NaN,NaN,2022-09-19,2022-09,2022Q3,1080.0,Medium,Medium,[1000-1400[,Piemonte,NO,[60-72],[5000-10000[,23,[21-31[,A6,INDEPENDENT,3,[3-5[ I,10,0,A7,0
2,CIT20220907LAEWR2D,FACILE,IT,H,PA,1974-10-18,14,ITALIAN,PALERMO,PA,EMPLOYEE,2500.0,2500.0,60,10000.0,PERMANENT,PRIVATE_SECTOR,NEWCAR,2022-09-07 14:12:18+00:00,pre_approved,canceled,8,9,12,2001-01-01,0.0799,0.0620,0.0799,0.0603,0.0998,9824,NaN,NaN,NaN,2022-09-07,2022-09,2022Q3,2500.0,Medium,Risk,>1800,Sicilia,IS,[60-72],[10000-15000[,47,[31-51[,A3,PERMANENT,21,>=8 P,6,0,A4,0
3,CIT20221031BXKB27E,FACILE,IT,I,BO,1995-09-18,14,ITALIAN,BOLOGNA,BO,EMPLOYEE,2100.0,NaN,84,35000.0,PERMANENT,PRIVATE_SECTOR,LIQUIDITY,2022-10-31 07:57:45+00:00,pre_approved,None,12,9,14,2021-10-01,0.0949,0.0888,0.0949,0.0854,0.1240,9726,NaN,NaN,NaN,2022-10-31,2022-10,2022Q4,2100.0,Medium,Medium,>1800,Emilia-Romagna,CE,[72-84],>30000,27,[21-31[,A4,PERMANENT,1,[0-2[ P,9,0,A5,0
4,CIT20221011CA3XLVT,FACILE,IT,N,PT,2000-01-17,13,ITALIAN,PISTOIA,PO,WORKER,1400.0,1391.0,36,5000.0,PERMANENT,PRIVATE_SECTOR,USEDCAR,2022-10-11 08:11:37+00:00,pre_approved,granted,15,7,13,2020-06-01,0.0535,0.1232,0.0535,0.1167,0.1669,9579,0.0,0.0,0.0,2022-10-11,2022-10,2022Q4,1400.0,Safe,Safe,[1400-1800[,Toscana,CE,[36-48],[5000-10000[,22,[21-31[,A6,PERMANENT,2,[2-4[ P,10,1,A7,0


In [4]:
row = ['application_date_yymm']
col = ['business_context_business_provider_code','Cutoff_new','Cutoff_new_v1','quality_score','Rip_geo','Region','Experian_score_GRP','Profession_mapping',
       'Income_GRP','Borrowed_Amount_GRP','Age_GRP','Maturity_GRP',
       'Seniority_GRP','product_loan_initial_demand_project_type_code']
value = ['conversion']



df2 = df.copy()
df2 = df2[(df2['application_date']>=start_date) & 
          (df2['application_date']<end_date)]
df2 = df2[df2.business_context_business_provider_code == 'PRESTITIONLINE']

total = [row,col]
combinations = list(itertools.product(*total))

sort_by_number_obs = False
dic_output = {}


for lists in combinations:
        r = lists[0]
        c = lists[1]
        if c==r:
            continue


        pivot2 = pd.pivot_table(df2, values=value, 
                                        index=[r], 
                                        columns=[c],
                                        aggfunc='mean',
                                        fill_value=0, 
                                        margins=True, 
                                        margins_name='Total').style.format('{:.1%}')

        
        pivot_CIT = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total',normalize= 'index')
        pivot_n = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total')
                          

        print('============== conversion ==============')
        display(pivot2)
        print('============== SHARE IN # ==============')
        display(pivot_CIT.applymap(lambda x: "{0:.1f}%".format(100*x)) )    
        display(pivot_n)
        print()
        print()

============== conversion ==============


============== SHARE IN # ==============


business_context_business_provider_code,PRESTITIONLINE
application_date_yymm,
2022-07,100.0%
2022-08,100.0%
2022-09,100.0%
2022-10,100.0%
2022-11,100.0%
Total,100.0%


business_context_business_provider_code,PRESTITIONLINE,Total
application_date_yymm,,
2022-07,1364,1364
2022-08,813,813
2022-09,1283,1283
2022-10,1771,1771
2022-11,1212,1212
Total,6443,6443




============== conversion ==============


...

Seniority_GRP,>=10 I,>=8 P,>=8 others,[0-2[ P,[0-2[ others,[2-4[ P,[2-4[ others,[3-5[ I,[4-8[ P,[4-8[ others,[5-7[ I,[7-10[ I,Total
application_date_yymm,,,,,,,,,,,,,
2022-07,1.0,486.0,94.0,213.0,26.0,240.0,25.0,1.0,239.0,39.0,NaN,NaN,1364
2022-08,1.0,359.0,65.0,NaN,19.0,152.0,15.0,NaN,172.0,30.0,NaN,NaN,813
2022-09,3.0,616.0,97.0,3.0,23.0,226.0,13.0,NaN,273.0,29.0,NaN,NaN,1283
2022-10,105.0,709.0,135.0,2.0,25.0,268.0,37.0,56.0,340.0,39.0,23.0,32.0,1771
2022-11,84.0,480.0,93.0,NaN,20.0,196.0,20.0,NaN,232.0,28.0,28.0,31.0,1212
Total,194.0,2650.0,484.0,218.0,113.0,1082.0,110.0,57.0,1256.0,165.0,51.0,63.0,6443




============== conversion ==============


============== SHARE IN # ==============


product_loan_initial_demand_project_type_code,APPLIANCES,DEBTS,FURNITURE,HITECH,HOMEIMPROVEMENT,LIQUIDITY,MEDICALCARE,NEWCAR,TRAINING,USEDCAR,VACATION,WEDDINGS
application_date_yymm,,,,,,,,,,,,
2022-07,0.4%,11.1%,6.4%,0.1%,18.6%,27.1%,3.1%,6.1%,0.1%,24.6%,2.0%,0.5%
2022-08,0.2%,12.8%,4.8%,0.2%,20.7%,27.8%,2.2%,6.0%,0.7%,22.0%,1.6%,0.9%
2022-09,0.3%,15.1%,5.1%,0.1%,17.9%,30.5%,3.7%,6.0%,0.5%,19.2%,0.9%,0.8%
2022-10,0.1%,14.5%,4.2%,0.2%,15.3%,34.8%,3.0%,5.9%,0.7%,19.9%,0.5%,0.9%
2022-11,0.1%,13.4%,4.8%,0.4%,15.8%,34.6%,1.7%,6.4%,0.3%,20.9%,1.2%,0.6%
Total,0.2%,13.5%,5.0%,0.2%,17.3%,31.4%,2.8%,6.1%,0.5%,21.2%,1.1%,0.7%


product_loan_initial_demand_project_type_code,APPLIANCES,DEBTS,FURNITURE,HITECH,HOMEIMPROVEMENT,LIQUIDITY,MEDICALCARE,NEWCAR,TRAINING,USEDCAR,VACATION,WEDDINGS,Total
application_date_yymm,,,,,,,,,,,,,
2022-07,5,151,87,1,254,369,42,83,2,336,27,7,1364
2022-08,2,104,39,2,168,226,18,49,6,179,13,7,813
2022-09,4,194,66,1,230,391,47,77,6,246,11,10,1283
2022-10,2,256,75,4,271,617,53,104,12,353,8,16,1771
2022-11,1,162,58,5,191,419,20,78,4,253,14,7,1212
Total,14,867,325,13,1114,2022,180,391,30,1367,73,47,6443


In [11]:
row = ['product_loan_initial_demand_maturity']
col = ['Borrowed_Amount_GRP']
value = ['conversion']



df2 = df.copy()
df2 = df2[(df2['application_date']>=start_date) & 
          (df2['application_date']<end_date)]
df2 = df2[df2.business_context_business_provider_code == 'PRESTITIONLINE']

total = [row,col]
combinations = list(itertools.product(*total))

sort_by_number_obs = False
dic_output = {}


for lists in combinations:
        r = lists[0]
        c = lists[1]
        if c==r:
            continue


        pivot2 = pd.pivot_table(df2, values=value, 
                                        index=[r], 
                                        columns=[c],
                                        aggfunc='mean',
                                        fill_value=0, 
                                        margins=True, 
                                        margins_name='Total').style.format('{:.1%}')

        
        pivot_CIT = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total',normalize= 'index')
        pivot_n = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total')
                          

        print('============== conversion ==============')
        display(pivot2)
        print('============== SHARE IN # ==============')
        display(pivot_CIT.applymap(lambda x: "{0:.1f}%".format(100*x)) )    
        display(pivot_n)
        print()
        print()

============== conversion ==============


============== SHARE IN # ==============


Borrowed_Amount_GRP,>30000,[1000-5000[,[10000-15000[,[15000-20000[,[20000-30000[,[5000-10000[
product_loan_initial_demand_maturity,,,,,,
36,1.2%,29.5%,20.5%,4.1%,1.9%,42.7%
48,2.2%,19.7%,26.0%,9.6%,5.9%,36.7%
60,6.2%,10.6%,35.3%,11.1%,10.9%,25.9%
72,9.3%,6.2%,28.5%,21.4%,17.4%,17.2%
84,34.5%,3.8%,18.2%,14.9%,17.7%,10.9%
Total,10.2%,12.4%,28.7%,11.9%,11.1%,25.8%


Borrowed_Amount_GRP,>30000,[1000-5000[,[10000-15000[,[15000-20000[,[20000-30000[,[5000-10000[,Total
product_loan_initial_demand_maturity,,,,,,,
36,10,243,169,34,16,352,824
48,16,146,193,71,44,273,743
60,189,322,1074,338,333,789,3045
72,69,46,212,159,129,128,743
84,375,41,198,162,193,119,1088
Total,659,798,1846,764,715,1661,6443


In [17]:
row = ['product_loan_initial_demand_maturity']
col = ['Borrowed_Amount_GRP']
value = ['Macro_A']



df2 = df.copy()
df2 = df2[(df2['application_date']>=start_date) & 
          (df2['application_date']<end_date)]
df2 = df2[df2.business_context_business_provider_code == 'PRESTITIONLINE']
df2 = df2[df2.process_decisions_timeline_decision_status == 'granted']

total = [row,col]
combinations = list(itertools.product(*total))

sort_by_number_obs = False
dic_output = {}


for lists in combinations:
        r = lists[0]
        c = lists[1]
        if c==r:
            continue


        pivot2 = pd.pivot_table(df2, values=value, 
                                        index=[r], 
                                        columns=[c],
                                        aggfunc='mean',
                                        fill_value=0, 
                                        margins=True, 
                                        margins_name='Total').style.format('{:.1%}')

        
        pivot_CIT = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total',normalize= 'index')
        pivot_n = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total')
                          

        print('============== Macro-A ==============')
        display(pivot2)
        print('============== SHARE IN # ==============')
        display(pivot_CIT.applymap(lambda x: "{0:.1f}%".format(100*x)) )    
        display(pivot_n)
        print()
        print()

============== Macro-A ==============


============== SHARE IN # ==============


Borrowed_Amount_GRP,>30000,[1000-5000[,[10000-15000[,[15000-20000[,[20000-30000[,[5000-10000[
product_loan_initial_demand_maturity,,,,,,
36,0.8%,23.5%,24.2%,6.8%,0.8%,43.9%
48,0.6%,17.1%,25.0%,8.5%,7.9%,40.9%
60,2.1%,8.6%,36.6%,9.9%,11.8%,31.0%
72,3.6%,8.6%,33.6%,20.0%,8.6%,25.7%
84,15.6%,5.6%,26.9%,14.4%,20.6%,16.9%
Total,4.1%,11.5%,30.9%,11.4%,10.6%,31.3%


Borrowed_Amount_GRP,>30000,[1000-5000[,[10000-15000[,[15000-20000[,[20000-30000[,[5000-10000[,Total
product_loan_initial_demand_maturity,,,,,,,
36,1,31,32,9,1,58,132
48,1,28,41,14,13,67,164
60,8,32,137,37,44,116,374
72,5,12,47,28,12,36,140
84,25,9,43,23,33,27,160
Total,40,112,300,111,103,304,970


In [8]:
df2.head(5)


,contract_reference,business_context_business_provider_code,business_unit_code,partner_creditbureau_crif_last_call_perform_score,contact_public_current_address_province_code,contact_private_declared_birth_date,situation_declared_number_of_paid_months_per_year,contact_public_declared_nationality_code,contact_public_declared_address_town_label,contact_public_declared_province_code,situation_declared_profession_code,situation_declared_main_net_monthly_income,situation_verified_main_net_monthly_income,product_loan_initial_demand_maturity,product_loan_initial_demand_requested_amount,situation_declared_professional_situation_code,situation_declared_profession_activity_area_code,product_loan_initial_demand_project_type_code,process_decisions_timeline_application_date,process_decisions_timeline_pre_decision_status,process_decisions_timeline_decision_status,partner_creditbureau_experian_last_call_delphi_g3_index1,partner_creditbureau_experian_last_call_delphi_g3_index3,partner_creditbureau_experian_last_call_cdi,situation_declared_profession_start_date,product_loan_selected_application_fee_rate,product_loan_selected_lending_rate,product_loan_selected_application_fee_rate_1,product_loan_selected_nominal_annual_rate,product_loan_selected_annual_percentage_rate,process_decisions_credit_scoring_declared_score_value,credit,mortgage,rent,application_date,application_date_yymm,application_date_in_quarter,Corrected_Income,Crif_score_GRP,Experian_score_GRP,Income_GRP,Region,Rip_geo,Maturity_GRP,Borrowed_Amount_GRP,AGE,Age_GRP,Cutoff,Profession_mapping,Seniority,Seniority_GRP,quality_score,conversion,Cutoff_new,Cutoff_new_v1
1,CIT20220919AIIZIIA,FACILE,IT,I,TO,1998-09-24,13,ITALIAN,GRUGLIASCO,TO,WORKER,1080.0,1080.0,72,7000.0,INDEPENDENT,None,USEDCAR,2022-09-19 07:00:52+00:00,pre_approved,canceled,14,7,14,2019-07-01,0.0102,0.1534,0.0102,0.1436,0.1579,9553,NaN,NaN,NaN,2022-09-19,2022-09,2022Q3,1080.0,Medium,Medium,[1000-1400[,Piemonte,NO,[60-72],[5000-10000[,23,[21-31[,A6,INDEPENDENT,3,[3-5[ I,10,0,A7,A7
7,CIT20221111SMRM14L,FACILE,IT,N,PA,1990-04-16,<NA>,ITALIAN,PALERMO,CT,INSURANCEAGENT,2500.0,3039.0,60,5000.0,INDEPENDENT,None,DEBTS,2022-11-11 19:06:23+00:00,pre_approved,granted,11,7,15,2012-11-01,0.0883,0.1202,0.0883,0.1140,0.1669,9702,376.0,0.0,0.0,2022-11-11,2022-11,2022Q4,2500.0,Safe,Medium,>1800,Sicilia,IS,[60-72],[5000-10000[,32,[31-51[,A4,INDEPENDENT,10,>=10 I,9,1,A5,A5
50,CIT20220525TAZCREY,FACILE,IT,H,NA,1972-01-01,<NA>,ITALIAN,TORRE ANNUNZIATA,NA,ARTISAN,5000.0,584.0,60,20000.0,INDEPENDENT,None,LIQUIDITY,2022-05-25 19:33:16+00:00,pre_approved,rejected,10,7,10,2016-05-01,0.0743,0.1061,0.0743,0.1013,0.1439,9323,NaN,NaN,NaN,2022-05-25,2022-05,2022Q2,5000.0,Medium,Medium,>1800,Campania,SU,[60-72],>20000,50,[31-51[,A7,INDEPENDENT,6,[5-7[ I,8,0,A7,A7
58,CIT20221104C2QWPV0,FACILE,IT,L,CT,1971-04-25,<NA>,ITALIAN,MISTERBIANCO,NO,INSURANCEAGENT,2100.0,NaN,60,15000.0,INDEPENDENT,None,HOMEIMPROVEMENT,2022-11-04 08:03:23+00:00,pre_approved,None,8,7,6,2001-06-01,0.0423,0.1446,0.0423,0.1358,0.1669,9681,NaN,NaN,NaN,2022-11-04,2022-11,2022Q4,2100.0,Medium,Risk,>1800,Sicilia,IS,[60-72],[10000-20000[,51,Missing,A5,INDEPENDENT,21,>=10 I,6,0,A6,A6
115,CIT20220916KAHW6CD,FACILE,IT,M,RE,1991-02-22,14,ITALIAN,CORREGGIO,None,EMPLOYEE,1400.0,1400.0,72,20000.0,INDEPENDENT,None,LIQUIDITY,2022-09-16 13:32:30+00:00,pre_approved,canceled,10,7,10,2015-05-01,0.0950,0.0965,0.0950,0.0925,0.1376,9722,NaN,NaN,NaN,2022-09-16,2022-09,2022Q3,1400.0,Safe,Medium,[1400-1800[,Emilia-Romagna,CE,[60-72],>20000,31,[31-51[,A4,INDEPENDENT,7,[7-10[ I,8,0,A5,A5


In [5]:
        
from IPython.html import widgets
from IPython.display import display
dic = {}
for x in col:
    dic[x] = df[x].unique().tolist()

def print_column(col):
    print(f' +++++ variable selected: {col} +++++')

def select_variable(var):
    variables.options = bpc[var]

def select_variable_2(var):
    variables_2.options = dic[var]

bpc = {}
bpc['business_provider_code'] = df.business_context_business_provider_code.unique().tolist()
cols = widgets.Select(options=bpc.keys())
init = cols.value
variables = widgets.Select(options=bpc[init])
j = widgets.interactive(print_column, col=variables)
i = widgets.interactive(select_variable, var=cols)
display(i)
display(j) 
#Aggiungerci tabella per scegliere i vari DR, tabella per fare filtro intersezione
filter_2 = 'Y'
if filter_2 == 'Y':
    cols_2 = widgets.Select(options=dic.keys())
    init_2 = cols_2.value
    variables_2 = widgets.Select(options=dic[init_2])
    j_2 = widgets.interactive(print_column, col=variables_2)
    i_2 = widgets.interactive(select_variable_2, var=cols_2)
    display(i_2)
    display(j_2) 


/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


interactive(children=(Select(description='var', options=('business_provider_code',), value='business_provider_…

interactive(children=(Select(description='col', options=('PRESTITIONLINE', 'FACILE', 'PRESTITIONLINE-PROMO', '…

interactive(children=(Select(description='var', options=('business_context_business_provider_code', 'Cutoff_ne…

interactive(children=(Select(description='col', options=('PRESTITIONLINE', 'FACILE', 'PRESTITIONLINE-PROMO', '…

In [6]:
bpc_name = variables.value
col_name = cols_2.value
risky_var = variables_2.value
print(f'BPC: {bpc_name} \nVariable to exclude, to get new conversion: {risky_var} \n\n')

df2 = df.copy()
df2 = df2[(df2['application_date']>=start_date) & 
          (df2['application_date']<end_date)]
df2 = df2[df2.business_context_business_provider_code == bpc_name]

conv_1 = df2.conversion.mean()
n_granted_1 = df2.conversion.sum()

df2 = df2[df2[col_name]!=risky_var]

conv_2 =  df2.conversion.mean()
n_granted_2 = df2.conversion.sum()

print(f'Actual conversion: {np.round(conv_1*100,2)} %')
print(f'New conversion: {np.round(conv_2*100,2)} %')
print(f'\nDelta conversion: {np.round((conv_2/conv_1-1)*100,2)} %')
print(f'Delta Granted #: {np.round((n_granted_2/n_granted_1-1)*100,2)} %')


BPC: PRESTITIONLINE 
Variable to exclude, to get new conversion: [21-31[ 


Actual conversion: 15.06 %
New conversion: 15.89 %

Delta conversion: 5.53 %
Delta Granted #: -5.46 %


In [ ]:
conversion_

Cutoff_new,A1,A2,A3,A4,A5,A6,A7,Missing,Total
application_date_yymm,,,,,,,,,
2022-07,0.202247,0.293860,0.207792,0.162304,0.068493,0.102041,0.023810,0,0.096491
2022-08,0.340909,0.236364,0.243243,0.111111,0.067797,0.186047,0.035461,0,0.106305
2022-09,0.379032,0.231920,0.173228,0.138577,0.086538,0.088235,0.026012,0,0.101133
2022-10,0.351648,0.224335,0.217822,0.059783,0.040541,0.130435,0.021113,0,0.084806
Total,0.323980,0.243705,0.205805,0.120647,0.067742,0.121359,0.026168,0,0.097094


In [12]:
row = ['application_date_yymm']
col = ['Cutoff_new']
value = ['conversion']



df2 = df.copy()
df2 = df2[(df2['application_date']>=start_date) & 
          (df2['application_date']<end_date)]
# df2 = df2[df2.Cutoff_new != 'Missing']
df2 = df2[df2.business_context_business_provider_code == 'PRESTITIONLINE']

total = [row,col]
combinations = list(itertools.product(*total))

sort_by_number_obs = False
dic_output = {}

conversion_ = pd.pivot_table(df2, values='conversion', 
                       index=['application_date_yymm'], 
                       columns=['Cutoff_new'],
                       aggfunc='mean',
                       fill_value=0, 
                       margins=True, 
                       margins_name='Total')

current_share = pd.crosstab(df2['application_date_yymm'], 
    df2['Cutoff_new'], values=df2['contract_reference'],aggfunc = 'count'
    ,margins=True,margins_name='Total',normalize= 'index')

new_share = pd.crosstab(df2['application_date_yymm'], 
    df2['Cutoff_new_v1'], values=df2['contract_reference'],aggfunc = 'count'
    ,margins=True,margins_name='Total',normalize= 'index')

conversion_list = conversion_.loc['Total'].values.tolist()[:-1]

conversion_list[-2] = 0
current_share_list = current_share.loc['Total'].values.tolist()[:-1]
new_share_list = new_share.loc['Total'].values.tolist()[:-1]

initial_preacc = np.round(sum(x * y for x, y in zip(conversion_list, current_share_list)),6)
result = np.round(sum(x * y for x, y in zip(conversion_list, new_share_list)),6)

print(f'Current conversion: {np.round(initial_preacc*100,2)} %')
print(f'New conversion: {np.round(result*100,2)} %')

KeyError: ignored

In [ ]:
df[(df.Profession_mapping == 'INDEPENDENT')&(df.application_date >= '2022-07-01')&(df.process_decisions_timeline_decision_status == 'granted')&].filter(regex='contract|provider|Senio')

,contract_reference,business_context_business_provider_code,Seniority,Seniority_GRP
1163,CIT20221021POIPM1D,FACILE,30,>=10 I
1755,CIT20220712BMOWKSM,PRESTITIONLINE-PROMO,3,[3-5[ I
1844,CIT20221012IKZPO0W,FACILE,8,[7-10[ I
12108,CIT20220923AIKB1S8,FACILE,17,>=10 I
23855,CIT20221008PL7N2DU,FACILE,3,[3-5[ I
30617,CIT20220912EHN6XNM,FACILE,15,>=10 I
30635,CIT20220912B9CHBCD,FACILE,24,>=10 I
31504,CIT20220818DNKT121,PRESTITIONLINE,30,>=10 I
32369,CIT20220706II9Q773,FACILE,0,[3-5[ I
33294,CIT20221006AI67AKV,FACILE,9,[7-10[ I


In [7]:
list_ = ['PRESTITIONLINE']

for x in list_:
    df2 = df[(df.business_context_business_provider_code == x) & 
             (df.Profession_mapping == 'INDEPENDENT')]
    value = "contract_reference"
    r = 'application_date_yymm'
    c = 'Seniority_GRP'

    pivot1 = pd.pivot_table(df2, values='conversion', 
                       index=['application_date_yymm'], 
                       columns=['Seniority_GRP'],
                       aggfunc='mean',
                       fill_value=0, 
                       margins=True, 
                       margins_name='Total')


    pivot2 = pd.crosstab(df2[r], 
        df2[c], values=df2['contract_reference'],aggfunc = 'count'
        ,margins=True,margins_name='Total')#,normalize= 'index')

    print(x)
    display(pivot1)
    print('=================================================')
    display(pivot2)

FACILE


Seniority_GRP,>=10 I,[3-5[ I,[5-7[ I,[7-10[ I,Total
application_date_yymm,,,,,
2022-01,0.059140,0.008403,0.035714,0.055556,0.034420
2022-02,0.076503,0.019900,0.032258,0.039216,0.044266
2022-03,0.075949,0.030075,0.035294,0.039216,0.048513
2022-04,0.038961,0.004673,0.039216,0.106383,0.030043
2022-05,0.136364,0.019139,0.081967,0.103448,0.074689
2022-06,0.106383,0.064748,0.045455,0.060606,0.076389
2022-07,0.000000,0.027778,0.000000,0.000000,0.019608
2022-08,0.000000,0.000000,0.000000,0.000000,0.000000
2022-09,0.095238,0.011905,0.000000,0.000000,0.032680


Seniority_GRP,>=10 I,[3-5[ I,[5-7[ I,[7-10[ I,Total
application_date_yymm,,,,,
2022-01,186,238,56,72,552
2022-02,183,201,62,51,497
2022-03,237,266,85,51,639
2022-04,154,214,51,47,466
2022-05,154,209,61,58,482
2022-06,94,139,22,33,288
2022-07,9,36,4,2,51
2022-08,1,40,2,2,45
2022-09,42,84,16,11,153


In [ ]:
list_ = ['FACILE']

df2 = df[(df.business_context_business_provider_code == x) & (df.Profession_mapping == 'INDEPENDENT') & (df.application_date_yymm > '2022-08') & (df.Seniority_GRP == '[3-5[ I')]
df3 = df2.contract_reference

df3.to_excel('Rocknroll.xlsx')

  
  

In [ ]:
row = ['Cutoff_new']
col = ['Maturity_GRP']
value = ['conversion']



df2 = df.copy()
df2 = df2[(df2['application_date']>=start_date) & 
          (df2['application_date']<end_date)]
df2 = df2[df2.business_context_business_provider_code == 'PRESTITIONLINE']

total = [row,col]
combinations = list(itertools.product(*total))

sort_by_number_obs = False
dic_output = {}


for lists in combinations:
        r = lists[0]
        c = lists[1]
        if c==r:
            continue


        pivot2 = pd.pivot_table(df2, values=value, 
                                        index=[r], 
                                        columns=[c],
                                        aggfunc='mean',
                                        fill_value=0, 
                                        margins=True, 
                                        margins_name='Total').style.format('{:.1%}')

        
        pivot_CIT = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total',normalize= 'index')
        pivot_n = pd.crosstab(df2[r], 
            df2[c], values=df2['contract_reference'],aggfunc = 'count'
            ,margins=True,margins_name='Total')
                          

        print('============== conversion ==============')
        display(pivot2)
        print('============== SHARE IN # ==============')
        display(pivot_CIT.applymap(lambda x: "{0:.1f}%".format(100*x)) )    
        display(pivot_n)
        print()
        print()

============== conversion ==============


============== SHARE IN # ==============


Maturity_GRP,[36-48],[60-72],[72-84]
Cutoff_new,,,
A1,26.6%,56.9%,16.5%
A2,27.6%,56.6%,15.9%
A3,24.7%,49.8%,25.5%
A4,23.0%,54.3%,22.7%
A5,25.3%,56.2%,18.5%
A6,27.6%,59.2%,13.3%
A7,21.5%,60.1%,18.4%
Missing,19.7%,63.6%,16.7%
Total,23.3%,58.8%,18.0%


Maturity_GRP,[36-48],[60-72],[72-84],Total
Cutoff_new,,,,
A1,121,259,75,455
A2,266,546,153,965
A3,67,135,69,271
A4,112,265,111,488
A5,45,100,33,178
A6,27,58,13,98
A7,204,570,175,949
Missing,278,896,235,1409
Total,1120,2829,864,4813


In [ ]:
pivot2.columns

MultiIndex([('conversion', '[36-48]'),
            ('conversion', '[60-72]'),
            ('conversion', '[72-84]'),
            ('conversion',   'Total')],
           names=[None, 'Maturity_GRP'])

In [ ]:
pivot2 = pd.pivot_table(df2, values=value, 
                                        index=[r], 
                                        columns=[c],
                                        aggfunc='mean',
                                        fill_value=0, 
                                        margins=True, 
                                       margins_name='Total')
display(pivot2)
# pivot2.drop("('conversion',   'Total')",axis=1)

conversion                              
Maturity_GRP    [36-48]   [60-72]   [72-84]     Total
Cutoff_new                                           
A1             0.452830  0.341667  0.342857  0.370192
A2             0.433884  0.379648  0.374150  0.393333
A3             0.483871  0.284553  0.216667  0.318367
A4             0.140000  0.155738  0.160377  0.153333
A5             0.073171  0.097826  0.031250  0.078788
A6             0.125000  0.111111  0.076923  0.109890
A7             0.021053  0.027132  0.023121  0.025028
Missing        0.000000  0.000000  0.000000  0.000000
Total          0.203340  0.144440  0.140073  0.157092

In [ ]:
pivot2.values#.tolist()

array([[0.44230769, 0.34745763, 0.33333333, 0.36919315],
       [0.44491525, 0.37401575, 0.38194444, 0.39414414],
       [0.48387097, 0.27642276, 0.21052632, 0.31404959],
       [0.14285714, 0.15899582, 0.15533981, 0.15454545],
       [0.075     , 0.0989011 , 0.03225806, 0.08024691],
       [0.13043478, 0.11538462, 0.07692308, 0.11363636],
       [0.01612903, 0.02750491, 0.02325581, 0.02422145],
       [0.        , 0.        , 0.        , 0.        ],
       [0.20481928, 0.1446297 , 0.13878563, 0.15723414]])

In [ ]:
for list_tab_1,list_tab_2 in zip(conversion.values,buf.values):
    for value1,value2 in zip(list_tab_1,list_tab_2):
           if value1 < 0.07 and value2 <0:
                

[0.44230769 0.34745763 0.33333333 0.36919315]
0.4423076923076923
[0.44230769 0.34745763 0.33333333 0.36919315]
0.3474576271186441
[0.44230769 0.34745763 0.33333333 0.36919315]
0.3333333333333333
[0.44230769 0.34745763 0.33333333 0.36919315]
0.3691931540342298
[0.44491525 0.37401575 0.38194444 0.39414414]
0.4449152542372881
[0.44491525 0.37401575 0.38194444 0.39414414]
0.37401574803149606
[0.44491525 0.37401575 0.38194444 0.39414414]
0.3819444444444444
[0.44491525 0.37401575 0.38194444 0.39414414]
0.39414414414414417
[0.48387097 0.27642276 0.21052632 0.31404959]
0.4838709677419355
[0.48387097 0.27642276 0.21052632 0.31404959]
0.2764227642276423
[0.48387097 0.27642276 0.21052632 0.31404959]
0.21052631578947367
[0.48387097 0.27642276 0.21052632 0.31404959]
0.3140495867768595
[0.14285714 0.15899582 0.15533981 0.15454545]
0.14285714285714285
[0.14285714 0.15899582 0.15533981 0.15454545]
0.1589958158995816
[0.14285714 0.15899582 0.15533981 0.15454545]
0.1553398058252427
[0.14285714 0.1589958